Data Source

https://www.kaggle.com/yelp-dataset/yelp-dataset

In [32]:
import findspark
findspark.init()
import pyspark


In [33]:
sc = pyspark.SparkContext(appName='big-data')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=big-data, master=local[*]) created by getOrCreate at <ipython-input-3-b5fd830c07f2>:2 

In [34]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('big-data').config("spark.driver.memory", "8g").config('spark.sql.execution.arrow.enabled', 'true').getOrCreate()

NameError: name 'sc' is not defined

## Reviews

In [5]:
reviews_df = spark.read.json('data/yelp_academic_dataset_review.json')

In [15]:
reviews_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



## Businesses

In [6]:
business_df = spark.read.json('data/yelp_academic_dataset_business.json')

In [17]:
business_df.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [7]:
reviews = reviews_df.alias('reviews')
business = business_df.alias('business')

In [35]:
from pyspark.sql.functions import concat_ws

examples = reviews.join(business, reviews.business_id == business.business_id) \
    .select(concat_ws(' ', reviews.stars, business.city, business.categories).alias('context'), \
            reviews.text.alias('review'))
    


In [20]:
examples.show()

+--------------------+--------------------+
|             context|              review|
+--------------------+--------------------+
|1.0 Las Vegas Fit...|Total bill for th...|
|5.0 Las Vegas Bea...|I *adore* Travis ...|
|5.0 Chandler Heal...|I have to say tha...|
|5.0 Calgary Bars,...|Went in for a lun...|
|1.0 Scottsdale Te...|Today was my seco...|
|4.0 Pittsburgh Re...|I'll be the first...|
|3.0 Markham Food,...|Tracy dessert had...|
|1.0 Scottsdale Sp...|This place has go...|
|2.0 Cleveland Bre...|I was really look...|
|3.0 Las Vegas Sho...|It's a giant Best...|
|4.0 Las Vegas Per...|Like walking back...|
|1.0 Mesa Restaura...|Walked in around ...|
|4.0 Pittsburgh It...|Wow. So surprised...|
|4.0 Las Vegas Hot...|Michael from Red ...|
|1.0 Toronto Asian...|I cannot believe ...|
|5.0 Toronto Sandw...|You can't really ...|
|4.0 Orange Villag...|Great lunch today...|
|3.0 Phoenix Carib...|I love chinese fo...|
|5.0 Chandler Sand...|We've been a huge...|
|3.0 Toronto Resta...|Good selec

DataFrame[context: string, review: string]

DataFrame[context: string, review: string]

Unfortunately, the default ML Tokenizer is really dumb, there is a RegexTokenizer but we could actually try to use an industrial-strength
Tokenizer from spacy.

Implementing an ML-Transformer would require a Java counter-part on top of the python implementation, so let's fall back to UDF's instead.

In [36]:
import spacy
from pyspark.sql.functions import udf, pandas_udf
from pyspark.sql.types import ArrayType, StringType

nlp = spacy.load("en_core_web_sm")

@udf(ArrayType(StringType()))
def tokenize(s):
    return [token.text for token in nlp(s)]

In [37]:
tokenized = examples.select(concat_ws(' ', tokenize(examples.context)).alias('context'), concat_ws(' ', tokenize(examples.review)).alias('review'))

In [21]:
tokenized.show()

+--------------------+--------------------+
|             context|              review|
+--------------------+--------------------+
|1.0 Las Vegas Fit...|Total bill for th...|
|5.0 Las Vegas Bea...|I * adore * Travi...|
|5.0 Chandler Heal...|I have to say tha...|
|5.0 Calgary Bars ...|Went in for a lun...|
|1.0 Scottsdale Te...|Today was my seco...|
|4.0 Pittsburgh Re...|I 'll be the firs...|
|3.0 Markham Food ...|Tracy dessert had...|
|1.0 Scottsdale Sp...|This place has go...|
|2.0 Cleveland Bre...|I was really look...|
|3.0 Las Vegas Sho...|It 's a giant Bes...|
|4.0 Las Vegas Per...|Like walking back...|
|1.0 Mesa Restaura...|Walked in around ...|
|4.0 Pittsburgh It...|Wow . So surprise...|
|4.0 Las Vegas Hot...|Michael from Red ...|
|1.0 Toronto Asian...|I can not believe...|
|5.0 Toronto Sandw...|You ca n't really...|
|4.0 Orange Villag...|Great lunch today...|
|3.0 Phoenix Carib...|I love chinese fo...|
|5.0 Chandler Sand...|We 've been a hug...|
|3.0 Toronto Resta...|Good selec

In [38]:
tokenized = tokenized.limit(1000)

In [39]:
tokenized = tokenized.cache()

In [ ]:
tokenized.count()

In [ ]:
tokenized.rdd.getNumPartitions()

In [22]:
sub = tokenized.limit(50)

In [26]:
sub.count()

50

In [23]:
pandas_df = sub.toPandas()

KeyboardInterrupt: 

In [ ]:
pandas_df

In [ ]:
#clean = sub.na.drop()

In [ ]:
#clean.count()

In [ ]:
#train, val, test = sub.randomSplit([0.9, 0.75, 0.25], seed=12)


In [ ]:
#train.count()
#val.count()
#test.count()

In [24]:
def write_df(df, file_name):
    df.coalesce(1).select(df.context).write.format('csv').save(file_name + '_src.txt')
    df.coalesce(1).select(df.review).write.format('csv').save(file_name + '_tgt.txt')
    


In [ ]:
#write_df(train, 'train')
#write_df(val, 'val')
#write_df(test, 'test')

In [25]:
write_df(sub, 'booyaka')

KeyboardInterrupt: 